In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/word2vec/GoogleNewsUnigrams.txt
/kaggle/input/corpus/outFile_pre.tsv
/kaggle/input/corpus/outFile_integration.tsv
/kaggle/input/corpus/outFile.tsv


In [2]:
!python --version

Python 3.10.13


In [3]:
import tensorflow as tf
print(tf.__version__)


2024-06-28 09:49:46.818693: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 09:49:46.818832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 09:49:46.993640: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd

def readData(dataFile,maxSequenceLength,embeddings):
	labels=[]
	sources=[]
	wordsD={'<PAD>':0,'<UNK>':1}
	wordsL=['<PAD>','<UNK>']
	contents=[]
	nums=[]
	documentCVs=[]
	topicCVs=[]
	sourceCVs=[]
	fileIn = open(dataFile, 'r', encoding='utf-8', errors='ignore')
	# fileIn=open(dataFile,'r')
	while True:
		line=fileIn.readline()
		if line=='':
			break
		parts=line.strip().split('\t')
		label=parts[0]
		source=parts[1]
		num=parts[2]
		documentCV=parts[3]
		topicCV=parts[4]
		sourceCV=parts[5]
		contentWords=parts[6:]
		if len(contentWords)>maxSequenceLength:
			contentWords=contentWords[0:maxSequenceLength]
		content=[]
		for contentWord in contentWords:
			if not contentWord in embeddings:
				contentWord='<UNK>'
			elif not contentWord in wordsD:
				wordsD[contentWord]=len(wordsD)
				wordsL.append(contentWord)
			content.append(wordsD[contentWord])
		if len(content)<maxSequenceLength:
			content=content+[0]*(maxSequenceLength-len(content))
		labels.append(label)
		sources.append(source)
		contents.append(content)
		nums.append(num)
		documentCVs.append(documentCV)
		topicCVs.append(topicCV)
		sourceCVs.append(sourceCV)
	fileIn.close()
	return(labels,sources,nums,wordsL,contents,documentCVs,topicCVs,sourceCVs)


def readDocuments(dataFile,maxSequenceLength,embeddings,maxDocumentLength):
	(labels,sources,nums,wordsL,contents,documentCVs,topicCVs,sourceCVs)=readData(dataFile,maxSequenceLength,embeddings)
	labelsD=[]
	sourcesD=[]
	numsD=[]
	contentsD=[]
	documentCVsD=[]
	topicCVsD=[]
	sourceCVsD=[]
	lengthsD=[]
	i=0
	contentHere=[]
	while(True):
		contentHere.append(contents[i])
		if i==len(nums)-1 or nums[i+1]!=nums[i]:
			labelsD.append(labels[i])
			sourcesD.append(sources[i])
			numsD.append(nums[i])
			documentCVsD.append(documentCVs[i])
			topicCVsD.append(topicCVs[i])
			sourceCVsD.append(sourceCVs[i])
			if len(contentHere)<maxDocumentLength:
				lengthHere=len(contentHere)
				for j in range(maxDocumentLength-len(contentHere)):
					contentHere.append([0]*maxSequenceLength)
			else:
				lengthHere=maxDocumentLength
				contentHere=contentHere[0:maxDocumentLength]
			lengthsD.append(lengthHere)
			contentsD.append(contentHere)
			contentHere=[]
		i=i+1
		if i==len(nums):
			break
	return(labelsD,sourcesD,numsD,wordsL,contentsD,documentCVsD,topicCVsD,sourceCVsD,lengthsD)


def readEmbeddings(vectorsFile):
    embeddings={}
    with open(vectorsFile, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.split()
            if len(parts) != 301:
                continue
            coefs = np.asarray(parts[1:], dtype='float32')
            embeddings[parts[0]] = coefs
    return embeddings
# def readEmbeddings(vectorsFile):
# 	embeddings={}
# 	for line in open(vectorsFile):
# 		parts=line.split()
# 		if len(parts)!=301:
# 			continue
# 		coefs=np.asarray(parts[1:],dtype='float32')
# 		embeddings[parts[0]]=coefs
# 	return(embeddings)

def prepareEmbeddings(embeddings,wordsL):
	embeddingMatrix = np.zeros((len(wordsL), 300))
	for i in range(len(wordsL)):
		row=np.zeros(300)
		word=wordsL[i]
		if word!='<PAD>' and word!='<UNK>':
			row=embeddings[word]
			row=row/np.sqrt(sum(row*row))
		embeddingMatrix[i]=row
	return(embeddingMatrix)

def lengthToAverageMask(lengthD,maxDocumentLength,binary=False):
	result=[]
	for i in range(len(lengthD)):
		if binary:
			multiplier=1.0
		else:
			multiplier=maxDocumentLength*1.0/lengthD[i]
		vec=[multiplier,multiplier]
		vec0=[0.0,0.0]
		vector=[vec]*lengthD[i]+[vec0]*(maxDocumentLength-lengthD[i])
		result.append(vector)
	return(np.array(result))


def evaluateD(resultPred,resultTrue):
	return(np.mean((resultPred[:,1]>0.5)==(resultTrue[:,1]==1)))



class Style1():
	def __init__(self,embeddingMatrix,labelsNum,maxSequenceLength,maxDocumentLength,onGPU):
		self.maxSequenceLength=maxSequenceLength
		self.maxDocumentLength=maxDocumentLength
		self.labelsNum=labelsNum
		self.embeddingSize=np.shape(embeddingMatrix)[1]
		self.embL=keras.layers.Embedding(np.shape(embeddingMatrix)[0],self.embeddingSize,input_length=maxSequenceLength,weights=[embeddingMatrix],trainable=False)
		self.reshape1L=keras.layers.Lambda(self.backend_reshape,output_shape=(maxSequenceLength,self.embeddingSize))
		if onGPU:
			self.LSTMforL=keras.layers.LSTM(units=100,go_backwards=False,return_sequences=False)
			self.LSTMrevL=keras.layers.LSTM(units=100,go_backwards=True,return_sequences=False)
		else:
			self.LSTMforL=keras.layers.LSTM(units=100,go_backwards=False,return_sequences=False)
			self.LSTMrevL=keras.layers.LSTM(units=100,go_backwards=True,return_sequences=False)
		self.conL=keras.layers.Concatenate(axis=1)
		self.denseL=keras.layers.Dense(labelsNum,activation="softmax")
		self.reshape2L=keras.layers.Lambda(self.backend_reshape2,output_shape=(maxDocumentLength,labelsNum))
		self.multiply=keras.layers.Multiply()
		self.poolingL=keras.layers.GlobalAveragePooling1D()

	def backend_reshape(self,x):
		return keras.backend.reshape(x,(-1,self.maxSequenceLength,self.embeddingSize))

	def backend_reshape2(self,x):
		return keras.backend.reshape(x,(-1,self.maxDocumentLength,self.labelsNum))

	def getMask(self,lengthD):
		return (lengthToAverageMask(lengthD,self.maxDocumentLength,binary=False))

	def getModel(self):
		inputWords = keras.layers.Input(shape=(self.maxDocumentLength,self.maxSequenceLength,))
		inputMask = keras.layers.Input(shape=(self.maxDocumentLength,self.labelsNum,))
		x=self.embL(inputWords)
		x=self.reshape1L(x)
		lstm1=self.LSTMforL(x)
		lstm2=self.LSTMrevL(x)
		x=self.conL([lstm1,lstm2])
		Ps=self.denseL(x)
		Ps=self.reshape2L(Ps)
		Ps=self.multiply([Ps,inputMask])
		Ps=self.poolingL(Ps)
		model=Model(inputs=[inputWords,inputMask], outputs=Ps)
		return(model)


dataPath="/kaggle/input/corpus"

batch_size=64
onGPU=False
epochs=10
dataset="outFile.tsv"
dataset_1="outFile_integration.tsv"

# Reading data
# exec(open('./functions.py').read())
embeddings=readEmbeddings("/kaggle/input/word2vec"+"/GoogleNewsUnigrams.txt")
MAX_SEQUENCE_LENGTH=120
MAX_DOCUMENT_LENGTH=50
(labels,sources,nums,wordsL,contents,documentCV,topicCV,sourceCV,lengthD)=readDocuments(dataPath+"/"+dataset,MAX_SEQUENCE_LENGTH,embeddings,MAX_DOCUMENT_LENGTH)
(labels_1,sources_1,nums_1,wordsL_1,contents_1,documentCV_1,topicCV_1,sourceCV_1,lengthD_1)=readDocuments(dataPath+"/"+dataset_1,MAX_SEQUENCE_LENGTH,embeddings,MAX_DOCUMENT_LENGTH)
embeddingMatrix=prepareEmbeddings(embeddings,wordsL)
embeddingMatrix_1=prepareEmbeddings(embeddings,wordsL_1)

# Converting to numpy
y=np.asarray(labels,dtype='float32')
y_1=np.asarray(labels_1,dtype='float32')
allY=np.concatenate((np.expand_dims(1-y,1),np.expand_dims(y,1)),axis=1)
allY_1=np.concatenate((np.expand_dims(1-y_1,1),np.expand_dims(y_1,1)),axis=1)
allX=np.array(contents)
allX_1=np.array(contents_1)

# Load models
# exec(open('./model.py').read())

# Prepare CV scenarios
# documentCV=np.asarray(documentCV,dtype='int32')
# topicCV=np.asarray(topicCV,dtype='int32')
sourceCV=np.asarray(sourceCV,dtype='int32')
scenarioCV=sourceCV
sourceCV_1=np.asarray(sourceCV_1,dtype='int32')
scenarioCV_1=sourceCV_1
result=np.array([[-1,-1]]*len(scenarioCV_1),dtype='float32')
fold=3
# print("Evaluating on fold "+str(fold)+"...")
whichTest=np.isin(scenarioCV,fold)
trainY=allY[~whichTest,]
develY=allY[whichTest,]
trainX=allX[~whichTest,]
develX=allX[whichTest,]
style1=Style1(embeddingMatrix,2,MAX_SEQUENCE_LENGTH,MAX_DOCUMENT_LENGTH,onGPU)
mask=style1.getMask(lengthD)
mask_1=style1.getMask(lengthD_1)
trainM=np.array(mask)[~whichTest]
develM=np.array(mask)[whichTest]
model=style1.getModel()
model.compile(optimizer=Adam(),loss="binary_crossentropy",metrics=["accuracy"])
model.fit([trainX,trainM],trainY, epochs=epochs,validation_data=([develX,develM],develY),batch_size=batch_size)
# model.save("/kaggle/working"+"/my_model.h5")
testX=allX_1
testY=allY_1
testM=np.array(mask_1)
predictions=model.predict([testX,testM])
result=predictions




# print(result)
# 创建一个字典，字典的键是列名，值是列的数据
df = pd.DataFrame()
df['num'] = nums_1
df['label'] = labels_1
df['predicton'] = result[:, 1]

df.to_csv("/kaggle/working"+"/result_integration_co.csv", index=False, encoding='utf-8-sig')

evaluateD(result,allY_1)
print(evaluateD(result,allY_1))

# //co

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import random

def readData(dataFile,maxSequenceLength,embeddings):
	labels=[]
	sources=[]
	wordsD={'<PAD>':0,'<UNK>':1}
	wordsL=['<PAD>','<UNK>']
	contents=[]
	nums=[]
	documentCVs=[]
	topicCVs=[]
	sourceCVs=[]
	fileIn = open(dataFile, 'r', encoding='utf-8', errors='ignore')
	# fileIn=open(dataFile,'r')
	while True:
		line=fileIn.readline()
		if line=='':
			break
		parts=line.strip().split('\t')
		label=parts[0]
		source=parts[1]
		num=parts[2]
		documentCV=parts[3]
		topicCV=parts[4]
# 		sourceCV=parts[5]
		sourceCV=random.randint(0, 5)
		contentWords=parts[6:]
		if len(contentWords)>maxSequenceLength:
			contentWords=contentWords[0:maxSequenceLength]
		content=[]
		for contentWord in contentWords:
			if not contentWord in embeddings:
				contentWord='<UNK>'
			elif not contentWord in wordsD:
				wordsD[contentWord]=len(wordsD)
				wordsL.append(contentWord)
			content.append(wordsD[contentWord])
		if len(content)<maxSequenceLength:
			content=content+[0]*(maxSequenceLength-len(content))
		labels.append(label)
		sources.append(source)
		contents.append(content)
		nums.append(num)
		documentCVs.append(documentCV)
		topicCVs.append(topicCV)
		sourceCVs.append(sourceCV)
	fileIn.close()
	return(labels,sources,nums,wordsL,contents,documentCVs,topicCVs,sourceCVs)


def readDocuments(dataFile,maxSequenceLength,embeddings,maxDocumentLength):
	(labels,sources,nums,wordsL,contents,documentCVs,topicCVs,sourceCVs)=readData(dataFile,maxSequenceLength,embeddings)
	labelsD=[]
	sourcesD=[]
	numsD=[]
	contentsD=[]
	documentCVsD=[]
	topicCVsD=[]
	sourceCVsD=[]
	lengthsD=[]
	i=0
	contentHere=[]
	while(True):
		contentHere.append(contents[i])
		if i==len(nums)-1 or nums[i+1]!=nums[i]:
			labelsD.append(labels[i])
			sourcesD.append(sources[i])
			numsD.append(nums[i])
			documentCVsD.append(documentCVs[i])
			topicCVsD.append(topicCVs[i])
			sourceCVsD.append(sourceCVs[i])
			if len(contentHere)<maxDocumentLength:
				lengthHere=len(contentHere)
				for j in range(maxDocumentLength-len(contentHere)):
					contentHere.append([0]*maxSequenceLength)
			else:
				lengthHere=maxDocumentLength
				contentHere=contentHere[0:maxDocumentLength]
			lengthsD.append(lengthHere)
			contentsD.append(contentHere)
			contentHere=[]
		i=i+1
		if i==len(nums):
			break
	return(labelsD,sourcesD,numsD,wordsL,contentsD,documentCVsD,topicCVsD,sourceCVsD,lengthsD)


def readEmbeddings(vectorsFile):
    embeddings={}
    with open(vectorsFile, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.split()
            if len(parts) != 301:
                continue
            coefs = np.asarray(parts[1:], dtype='float32')
            embeddings[parts[0]] = coefs
    return embeddings
# def readEmbeddings(vectorsFile):
# 	embeddings={}
# 	for line in open(vectorsFile):
# 		parts=line.split()
# 		if len(parts)!=301:
# 			continue
# 		coefs=np.asarray(parts[1:],dtype='float32')
# 		embeddings[parts[0]]=coefs
# 	return(embeddings)

def prepareEmbeddings(embeddings,wordsL):
	embeddingMatrix = np.zeros((len(wordsL), 300))
	for i in range(len(wordsL)):
		row=np.zeros(300)
		word=wordsL[i]
		if word!='<PAD>' and word!='<UNK>':
			row=embeddings[word]
			row=row/np.sqrt(sum(row*row))
		embeddingMatrix[i]=row
	return(embeddingMatrix)

def lengthToAverageMask(lengthD,maxDocumentLength,binary=False):
	result=[]
	for i in range(len(lengthD)):
		if binary:
			multiplier=1.0
		else:
			multiplier=maxDocumentLength*1.0/lengthD[i]
		vec=[multiplier,multiplier]
		vec0=[0.0,0.0]
		vector=[vec]*lengthD[i]+[vec0]*(maxDocumentLength-lengthD[i])
		result.append(vector)
	return(np.array(result))


def evaluateD(resultPred,resultTrue):
	return(np.mean((resultPred[:,1]>0.5)==(resultTrue[:,1]==1)))



class Style1():
	def __init__(self,embeddingMatrix,labelsNum,maxSequenceLength,maxDocumentLength,onGPU):
		self.maxSequenceLength=maxSequenceLength
		self.maxDocumentLength=maxDocumentLength
		self.labelsNum=labelsNum
		self.embeddingSize=np.shape(embeddingMatrix)[1]
		self.embL=keras.layers.Embedding(np.shape(embeddingMatrix)[0],self.embeddingSize,input_length=maxSequenceLength,weights=[embeddingMatrix],trainable=False)
		self.reshape1L=keras.layers.Lambda(self.backend_reshape,output_shape=(maxSequenceLength,self.embeddingSize))
		if onGPU:
			self.LSTMforL=keras.layers.LSTM(units=100,go_backwards=False,return_sequences=False)
			self.LSTMrevL=keras.layers.LSTM(units=100,go_backwards=True,return_sequences=False)
		else:
			self.LSTMforL=keras.layers.LSTM(units=100,go_backwards=False,return_sequences=False)
			self.LSTMrevL=keras.layers.LSTM(units=100,go_backwards=True,return_sequences=False)
		self.conL=keras.layers.Concatenate(axis=1)
		self.denseL=keras.layers.Dense(labelsNum,activation="softmax")
		self.reshape2L=keras.layers.Lambda(self.backend_reshape2,output_shape=(maxDocumentLength,labelsNum))
		self.multiply=keras.layers.Multiply()
		self.poolingL=keras.layers.GlobalAveragePooling1D()

	def backend_reshape(self,x):
		return keras.backend.reshape(x,(-1,self.maxSequenceLength,self.embeddingSize))

	def backend_reshape2(self,x):
		return keras.backend.reshape(x,(-1,self.maxDocumentLength,self.labelsNum))

	def getMask(self,lengthD):
		return (lengthToAverageMask(lengthD,self.maxDocumentLength,binary=False))

	def getModel(self):
		inputWords = keras.layers.Input(shape=(self.maxDocumentLength,self.maxSequenceLength,))
		inputMask = keras.layers.Input(shape=(self.maxDocumentLength,self.labelsNum,))
		x=self.embL(inputWords)
		x=self.reshape1L(x)
		lstm1=self.LSTMforL(x)
		lstm2=self.LSTMrevL(x)
		x=self.conL([lstm1,lstm2])
		Ps=self.denseL(x)
		Ps=self.reshape2L(Ps)
		Ps=self.multiply([Ps,inputMask])
		Ps=self.poolingL(Ps)
		model=Model(inputs=[inputWords,inputMask], outputs=Ps)
		return(model)
    
# 	def getModel(self):# 消融实验，去掉lstm层
# 		inputWords = keras.layers.Input(shape=(self.maxDocumentLength,self.maxSequenceLength,))
# 		inputMask = keras.layers.Input(shape=(self.maxDocumentLength,self.labelsNum,))
# 		x=self.embL(inputWords)
# 		x=self.reshape1L(x)
# 		# 删除 LSTM 层
# 		# lstm1=self.LSTMforL(x)
# 		# lstm2=self.LSTMrevL(x)
# 		# x=self.conL([lstm1,lstm2])
# 		x = keras.layers.Flatten()(x)  # 将词向量展平
# 		Ps=self.denseL(x)
# 		Ps=self.reshape2L(Ps)
# 		Ps=self.multiply([Ps,inputMask])
# 		Ps=self.poolingL(Ps)
# 		model=Model(inputs=[inputWords,inputMask], outputs=Ps)
# 		return(model)



dataPath="/kaggle/input/corpus"

batch_size=64
onGPU=True
epochs=10
dataset="outFile_pre.tsv"

# Reading data
# exec(open('./functions.py').read())
embeddings=readEmbeddings("/kaggle/input/word2vec"+"/GoogleNewsUnigrams.txt")
MAX_SEQUENCE_LENGTH=120
MAX_DOCUMENT_LENGTH=50
(labels,sources,nums,wordsL,contents,documentCV,topicCV,sourceCV,lengthD)=readDocuments(dataPath+"/"+dataset,MAX_SEQUENCE_LENGTH,embeddings,MAX_DOCUMENT_LENGTH)
embeddingMatrix=prepareEmbeddings(embeddings,wordsL)

# Converting to numpy
y=np.asarray(labels,dtype='float32')
allY=np.concatenate((np.expand_dims(1-y,1),np.expand_dims(y,1)),axis=1)
allX=np.array(contents)

# Load models
# exec(open('./model.py').read())

# Prepare CV scenarios
documentCV=np.asarray(documentCV,dtype='int32')
topicCV=np.asarray(topicCV,dtype='int32')
sourceCV=np.asarray(sourceCV,dtype='int32')
scenarioCV=sourceCV

result=np.array([[-1,-1]]*len(scenarioCV),dtype='float32')
for folda in range(max(scenarioCV)):
	fold=folda+1
	print("Evaluating on fold "+str(fold)+"...")
	whichTest=np.isin(scenarioCV,fold)
	trainY=allY[~whichTest,]
	develY=allY[whichTest,]
	trainX=allX[~whichTest,]
	develX=allX[whichTest,]
	style1=Style1(embeddingMatrix,2,MAX_SEQUENCE_LENGTH,MAX_DOCUMENT_LENGTH,onGPU)
	mask=style1.getMask(lengthD)
	trainM=np.array(mask)[~whichTest]
	develM=np.array(mask)[whichTest]
	model=style1.getModel()
	# model.compile(optimizer=tf.train.AdamOptimizer(),loss="binary_crossentropy",metrics=["accuracy"])
	model.compile(optimizer=Adam(),loss="binary_crossentropy",metrics=["accuracy"])
	fit=model.fit([trainX,trainM],trainY, epochs=epochs,validation_data=([develX,develM],develY),batch_size=batch_size)
	predictions=model.predict([develX,develM])
	result[whichTest,:]=predictions

evaluateD(result,allY)
print(evaluateD(result,allY))
# 创建一个字典，字典的键是列名，值是列的数据
df = pd.DataFrame()
df['num'] = nums
df['label'] = labels
df['predicton'] = result[:,1]

df.to_csv("/kaggle/working/result_pre_style.csv", index=False, encoding='utf-8-sig')

# //new